In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 200000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Weight_Deacy = 1e-5
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个


    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import AttributeNetwork2
from my_net_3 import MetricNetwork
from my_net_3 import MetricNetwork2

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork2(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)

attribute_network.cuda(GPU) 
metric_network.cuda(GPU)

#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=20000 , gamma=0.5)
metric_network_scheduler = StepLR(metric_network_optim , step_size=20000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    test_accuracy2 = total_rewards/1.0/test_size
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy, test_accuracy2

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num =0
#writer = SummaryWriter()
for episode in range(EPISODE):

    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)


    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    
    
    attribute_network.zero_grad()
    metric_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()

    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy, zsl_accuracy2 = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy , gzsl_unseen_accuracy2= compute_accuracy_per_class(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy ,gzsl_seen_accuracy2 = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.0980, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.0893968271698886
gzsl: unseen=0.0000 , seen=0.0233 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(3.0036, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.1961256950655755
gzsl: unseen=0.0082 , seen=0.0984 , h=0.0151
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9428, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.43212506804648454
gzsl: unseen=0.0551 , seen=0.2858 , h=0.0924
____________________________________________________________________________________________________
episode: 600 loss tensor(2.8430, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 

episode: 5600 loss tensor(1.0823, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6964218807193714
gzsl: unseen=0.5318 , seen=0.7348 , h=0.6170
____________________________________________________________________________________________________
episode: 5800 loss tensor(2.0565, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6939884252670334
gzsl: unseen=0.5391 , seen=0.7300 , h=0.6202
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.9358, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6831234149190389
gzsl: unseen=0.5357 , seen=0.7405 , h=0.6217
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.9134, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
clas

episode: 11200 loss tensor(1.9423, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6314057455354228
gzsl: unseen=0.5233 , seen=0.7509 , h=0.6168
____________________________________________________________________________________________________
episode: 11400 loss tensor(1.5675, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.630617614883354
gzsl: unseen=0.5127 , seen=0.7631 , h=0.6133
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.8037, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.625496651994545
gzsl: unseen=0.4972 , seen=0.7473 , h=0.5971
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.4110, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
cl

episode: 16800 loss tensor(1.8974, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6215125007713367
gzsl: unseen=0.4857 , seen=0.7725 , h=0.5964
____________________________________________________________________________________________________
episode: 17000 loss tensor(1.8724, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6303995825045667
gzsl: unseen=0.4954 , seen=0.7605 , h=0.6000
____________________________________________________________________________________________________
episode: 17200 loss tensor(2.1639, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.622409933157674
gzsl: unseen=0.4889 , seen=0.7690 , h=0.5978
____________________________________________________________________________________________________
episode: 17400 loss tensor(1.5243, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
c

episode: 22400 loss tensor(2.1398, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6210929130363945
gzsl: unseen=0.4893 , seen=0.7614 , h=0.5957
____________________________________________________________________________________________________
episode: 22600 loss tensor(1.0602, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6091189508894721
gzsl: unseen=0.4844 , seen=0.7627 , h=0.5925
____________________________________________________________________________________________________
episode: 22800 loss tensor(0.5504, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6039141201016704
gzsl: unseen=0.4837 , seen=0.7697 , h=0.5941
____________________________________________________________________________________________________
episode: 23000 loss tensor(0.7971, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


...

episode: 95200 loss tensor(2.0142, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5845461589377413
gzsl: unseen=0.4588 , seen=0.7788 , h=0.5775
____________________________________________________________________________________________________
episode: 95400 loss tensor(1.7029, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5844451727826783
gzsl: unseen=0.4578 , seen=0.7753 , h=0.5757
____________________________________________________________________________________________________
episode: 95600 loss tensor(0.6771, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5845620044088113
gzsl: unseen=0.4585 , seen=0.7740 , h=0.5759
____________________________________________________________________________________________________
episode: 95800 loss tensor(1.8714, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num: 50


episode: 100800 loss tensor(1.5883, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5865578043585633
gzsl: unseen=0.4654 , seen=0.7791 , h=0.5827
____________________________________________________________________________________________________
episode: 101000 loss tensor(1.1366, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5836355622141088
gzsl: unseen=0.4621 , seen=0.7796 , h=0.5803
____________________________________________________________________________________________________
episode: 101200 loss tensor(0.4654, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5870240490434436
gzsl: unseen=0.4638 , seen=0.7798 , h=0.5816
____________________________________________________________________________________________________
episode: 101400 loss tensor(1.0119, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num:

episode: 106400 loss tensor(2.6304, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5818277416461017
gzsl: unseen=0.4564 , seen=0.7748 , h=0.5744
____________________________________________________________________________________________________
episode: 106600 loss tensor(1.5718, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5849273724422991
gzsl: unseen=0.4590 , seen=0.7750 , h=0.5766
____________________________________________________________________________________________________
episode: 106800 loss tensor(1.0446, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5845154859754811
gzsl: unseen=0.4596 , seen=0.7741 , h=0.5767
____________________________________________________________________________________________________
episode: 107000 loss tensor(1.2653, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num:

episode: 112000 loss tensor(1.1154, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5886304122503491
gzsl: unseen=0.4615 , seen=0.7755 , h=0.5786
____________________________________________________________________________________________________
episode: 112200 loss tensor(1.2308, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5871930202743285
gzsl: unseen=0.4607 , seen=0.7770 , h=0.5785
____________________________________________________________________________________________________
episode: 112400 loss tensor(0.6392, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5868315268634224
gzsl: unseen=0.4587 , seen=0.7779 , h=0.5771
____________________________________________________________________________________________________
episode: 112600 loss tensor(0.6316, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num:

KeyboardInterrupt: 